**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Obtenga los 5 registros con valores más pequeños en la tercera columna.

In [29]:
%%writefile input.txt
B   1999-08-28   14
E   1999-12-06   121
E   1993-07-21   17
C   1991-02-12   2
E   1995-04-25   161
A   1992-08-22   14
B   1999-06-11   12
E   1993-01-27   8
E   1999-09-10   11
E   1990-05-03   16
E   1994-02-14   101
A   1988-04-27   9
A   1990-10-06   10
E   1985-02-12   16
E   1998-09-14   7
B   1994-08-30   17
A   1997-12-15   13
B   1995-08-23   101
B   1998-11-22   13
B   1997-04-09   6
E   1993-12-27   181
E   1999-01-14   15
A   1992-09-19   18
B   1993-03-02   14
B   1999-10-21   131
A   1990-08-31   12
C   1994-01-25   10
E   1990-02-09   18
A   1990-09-26   5
A   1993-05-08   16
B   1995-09-06   141
E   1991-02-18   14
A   1993-01-11   141
A   1990-07-22   4
C   1994-09-09   151
C   1994-07-27   1
D   1990-10-10   151
A   1990-09-05   11
B   1991-10-01   151
A   1994-10-25   13

Overwriting input.txt


## Mapper

In [30]:
%%writefile mapper.py
#! /usr/bin/env python3
import sys
import itertools

class Mapper():
    def __init__(self,stream):
        self.stream=stream
    def emit(self,key,value):
        sys.stdout.write("{},{}\n".format(key,value))
    def __iter__(self):
        for line in self.stream:
            key=line.split(" ")[0]
            val=float(line.split(" ")[6])
            yield(key,val)
    def map(self):
        for key, val in self:
            self.emit(key=key,value=val)

if __name__ == "__main__": 
    mapper=Mapper(sys.stdin)
    mapper.map()

Overwriting mapper.py


## Reducer

In [31]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
import itertools

class Reducer():
    def __init__(self,stream):
        self.stream=stream
    def emit(self,key,value):
        sys.stdout.write("{},{}\n".format(key,value))
    def __iter__(self):
        for line in self.stream:
            key=line.split(",")[0]
            value=float(line.split(",")[1])
            yield(key,value)
    def reduce(self):
        lista=[]
        for key, group in itertools.groupby(self,lambda x:x[0]):
            for key, value in group:
                lista.append((key,value))
            lista.sort(key=lambda x: x[1],reverse=False)
            
        for i in lista[:5]:
            self.emit(key=i[0],value=i[1])
           
                

if __name__ == '__main__': 
    reducer=Reducer(sys.stdin)
    reducer.reduce()

Overwriting reducer.py


## Ejecución

In [32]:
%%bash
rm -rf output
STREAM=$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar
chmod +x mapper.py
chmod +x reducer.py
hadoop jar $STREAM -input input.txt -output output  -mapper mapper.py -reducer reducer.py
cat output/part-00000

C,1.0	
C,2.0	
A,4.0	
A,5.0	
B,6.0	
